In [5]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [6]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 1, 41)):
    idx = ['Pop', 'IncR', 'MorR', 'Reach_ACF_Vul1', 'Reach_ACF_Vul2', 'Yield_ACF_Vul', 'Yield_ACF_Vul_TP', 'PPV_ACF_Vul']
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MU': {'Scale': 0}, 
                                                         'VulACF': {'Coverage': k}}, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
        mss_stats.append({
            'Key': i,
            'Coverage': k,
            'Pop0': ms.Pop.iloc[0],
            'IncN': (ms.IncR * ms.Pop).sum(),
            'MorN': (ms.MorR * ms.Pop).sum(),
            'Reach_ACF_Vul1': (ms.Reach_ACF_Vul1 * ms.Pop).sum(), 
            'Reach_ACF_Vul2': (ms.Reach_ACF_Vul2 * ms.Pop).sum(), 
            'Yield_ACF_Vul': (ms.Yield_ACF_Vul * ms.Pop).sum(), 
            'Yield_ACF_Vul_TP': (ms.Yield_ACF_Vul_TP * ms.Pop).sum(),
            'PPV_ACF_Vul': (ms.Yield_ACF_Vul_TP * ms.Pop).sum() / (ms.Yield_ACF_Vul * ms.Pop).sum()
        })
    mss = pd.concat(mss)
    mss_stats = pd.DataFrame(mss_stats)
    return mss, mss_stats

In [7]:
for folder in ['dy_free', 'dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss, mss_stats = collect_vul(pars_post, n_collect=100)
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_free


/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

...

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va

/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid value encountered in double_scalars
  mea_acf['PPV_ACF_Vul'] = mea_acf['Yield_ACF_Vul_TP'] / mea_acf['Yield_ACF_Vul']
/home/jovyan/work/sim/dy/model_intv.py:113: RuntimeWarning: invalid va